In this notebook we look at the language model class and the lightning wrapper

In [1]:
#First we fix the relative imports
import os
import sys
# M
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
from modules.LanguageModels.LstmLanguageModel import LSTMLanguageModel

In [3]:
## Every language model should implement the interface defined in the base language model

#We first look at a small example lstm based language model

from daily_dialog.DialogTokenizer import get_daily_dialog_tokenizer
my_tokenizer = get_daily_dialog_tokenizer(tokenizer_location='../daily_dialog/tokenizer.json', )


#This imports a fresh language model
language_model = LSTMLanguageModel(my_tokenizer.get_vocab_size())


In [4]:
#We can give a context and ask to complete it
from utils.utils import encode
context = "Hi how are you today? [SEP]"
ids = encode(my_tokenizer, context)
print(ids)
completed_dialogue = language_model.complete_dialogue(ids)
print(completed_dialogue)


tensor([427, 219, 178, 127, 484,  33,   1])
tensor([ 427,  219,  178,  127,  484,   33,    1, 1324, 1324,  950,   81, 1785,
         600,  600, 1232,   98, 1581, 1819,   57, 1819,   57, 1510,  406,   81,
         647, 1421, 1324, 1421,  171,   81,  171, 1210,  403,   98, 1545, 1324,
        1421, 1324, 1315, 1421, 1819, 1315, 1210, 2079, 1613, 1383, 1819, 1819,
        1083, 1210,  950, 1673, 1232, 1090, 1090, 1210, 1691, 1495, 1819,   81,
        1819,  851, 1691,   81, 1090,  181,   98, 2041, 1210, 1210,  403,   81,
        1673, 1893,  950,   98, 1673, 1613, 1673,  171, 1090, 1210,  403, 1324,
        1232,   81, 1421,  851, 1210, 1324, 1232, 1324, 1819, 1673, 1017, 1613,
         851,   98, 1324,  851])


In [5]:
## we decode the completed dialogue to see its response, which mostly nonsense as it is not trained yet
from utils.utils import decode
decoded_completed_dialogue = decode(my_tokenizer, completed_dialogue)
print(decoded_completed_dialogue)

hi how are you today ? [SEP]wherewhere muso calurnurn annx 100 bedroom s bedroom s colleometoacisionwhereision haveo have barectxesterwhereisionwhereuesision bedroomues barurs receive abs bedroom bedroom cred bar mus quality annauraur bar gradu art bedroomo bedroomanc graduoaurchx according bar barecto quality decision musx quality receive quality haveaur barectwhere annoisionanc barwhere annwhere bedroom quality cor receiveancxwhereanc


In [6]:
### Loading a trained model, which you can see already gives a better result, it is locally consistent.
location = "../small_lm_test.pt"
trained_language_model = LSTMLanguageModel.load(location) 

In [7]:
ids = encode(my_tokenizer, context)
print(ids)
completed_dialogue = trained_language_model.complete_dialogue(ids)
print(completed_dialogue)
decoded_completed_dialogue = decode(my_tokenizer, completed_dialogue)
print(decoded_completed_dialogue)

tensor([427, 219, 178, 127, 484,  33,   1])
tensor([ 427,  219,  178,  127,  484,   33,    1,   47,  209, 1058,   16,  208,
          47,   73,   51,  229,  334,   47,   73,   51,  286, 1508,  155,   47,
         291,   18,    1,  213,   16,  176,  159,  127,  239,  127,  177,  243,
         148,  179, 1005,   33,    1,  484,   11,  302,   16,  312,   18,    1,
         427,   16,   47,   11,   51, 1010,  255,   18,  176,   11,   57,  931,
          33,    1,  612,   18,  153,  140,  255,  281,   16,   72, 1153,  262,
          16,   47,  776,  157,   47,  381,  128,  872,  631,   18,  155,  156,
         128,  731,   88,  153,  261, 1217,   82,  690,  157,  152,  178,  863,
         229,  919,   39,  492])
hi how are you today ? [SEP] i like coffee , but i ’ m not sure i ’ m very tired and i am . [SEP] yes , what do you think you can get in your weekend ? [SEP] today ' ve , too . [SEP] hi , i ' m having all . what ' s wrong ? [SEP] hello . is it all right , ‘ cash here , i thought tha

In [8]:
### We wrape the language model into a pytorch lightning module. This makes training convinient and adds some more high-level functions.
# But it also requires giving some extra information to the model such as the hyperparameters
from modules.pytorch_lightning.LightningLanguageModel import LightningLanguageModel
hparams = {
    "learning_rate": 0.01,
    "teacher_forcing": True,
    "freeze_language_ml": True
}

lightning_language_model = LightningLanguageModel(trained_language_model, my_tokenizer, hparams=hparams)


In [9]:
## But it gives us some nice options such as easy completion of dialogues

dialogues = ["Hi how are you? ", "what are you upto? [SEP]"]

print(lightning_language_model.complete_dialogues(dialogues, max_length=100))

["hi how are you ? i ' m leaving now . [SEP] do you know i can find the nearest iccc ? [SEP] yes , it ' s nice and bright . [SEP] i thought that you ’ re in work on . [SEP] but we need a lot of things for the statess . how much ? how was it , please ? [SEP] we should go back out if i suggested for a job and some workal as a as interesting as my laall of . [SEP] you ’", "what are you upto ? [SEP] i ' m going to be fitstle on this night and relate is shopping of my school . [SEP] do you mind help ? [SEP] well , i like the delignc . i could be late . [SEP] then i am glad with my shoppings ! [SEP] what do you want to do ? [SEP] no , i ' ll take it on time , i know . [SEP] oh , hi . it ' s a good way , it ' s not like"]
